In [1]:
import numpy as np
import gurobipy as grb

In [2]:
X=Y=10
B=1000
np.random.seed(7)
phi_x_y = np.random.uniform(size = (X,Y))
epsilon_b_x_y = np.random.normal(size= (B,X,Y))
epsilon_b_x_0 = np.random.normal(size= (B,X))
eta_x_b_y =  np.random.normal(size= (X,B,Y))
eta_0_b_y =  np.random.normal(size= (B,Y))

In [3]:
m = grb.Model()
u_b_x = m.addMVar((B,X) , lb = -grb.GRB.INFINITY)
v_b_y = m.addMVar((B,Y), lb = -grb.GRB.INFINITY)
U_x_y = m.addMVar((X,Y), lb = -grb.GRB.INFINITY)
V_x_y = m.addMVar((X,Y), lb = -grb.GRB.INFINITY)
m.setObjective(u_b_x.sum() + v_b_y.sum(), sense = grb.GRB.MINIMIZE)
mu_x_y = m.addConstr(U_x_y + V_x_y == phi_x_y )
mu_b_x_y = m.addConstr(u_b_x[:,:,None] >= U_x_y[None,:,:] + epsilon_b_x_y )
mu_b_x_0 = m.addConstr(u_b_x >= epsilon_b_x_0 )
mu_x_b_y = m.addConstr(v_b_y[None,:,:] >= V_x_y[:,None,:] + eta_x_b_y )
mu_0_b_y = m.addConstr(v_b_y >= eta_0_b_y)


Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-08


In [4]:
m.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 220100 rows, 20200 columns and 420200 nonzeros
Model fingerprint: 0xb70d2100
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-06, 5e+00]
Presolve removed 20100 rows and 100 columns
Presolve time: 1.77s
Presolved: 20100 rows, 220000 columns, 420000 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.16s

Barrier statistics:
 AA' NZ     : 2.000e+05
 Factor NZ  : 2.256e+05 (roughly 100 MB of memory)
 Factor Ops : 2.782e+06 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual       

In [5]:
mu_x_y.pi

array([[ 65., 117., 101.,  98., 125.,  94.,  96.,  56.,  81.,  97.],
       [ 98., 117.,  92.,  69.,  83., 140.,  68.,  77., 120.,  57.],
       [ 93., 125.,  79.,  82., 112.,  60.,  86., 102., 106., 102.],
       [ 66.,  86.,  95.,  91.,  80., 135., 122.,  78.,  90.,  80.],
       [ 90.,  75., 125.,  84.,  77., 114.,  84., 122.,  61., 105.],
       [ 96.,  75., 129.,  96.,  81.,  49.,  69., 123., 101., 119.],
       [119.,  88.,  60.,  81.,  89.,  70., 138., 107., 119.,  82.],
       [134.,  74.,  95.,  90., 123.,  98.,  76.,  81.,  83.,  87.],
       [ 65.,  77.,  84., 100., 109.,  91.,  93., 102., 120., 110.],
       [111., 127.,  63., 139.,  70.,  84., 101., 101.,  62., 108.]])

In [6]:
n_x,m_y = np.ones(X),np.ones(Y)
a_x,b_y = np.zeros(X),np.zeros(Y)
K_x_y = np.exp(phi_x_y / 2)
maxIter = 10000
eps = 0.2
tol = 1e-5
for i in range(maxIter):
    mu_x_y = K_x_y * np.exp( (a_x[:,None]+b_y[None,:])/2 )
    mu_x0 = np.exp(a_x)
    mu_0y = np.exp(b_y)
    der_x = mu_x0 + mu_x_y.sum( axis = 1) - n_x 
    der_y = mu_0y + mu_x_y.sum( axis = 0) - m_y
    if max(np.max(np.abs(der_x / n_x)),np.max(np.abs(der_y / m_y)))<tol:
        break
    a_x -= eps * der_x
    b_y -= eps * der_y

print(i)
print(a_x,b_y)

73
[-2.62078988 -2.61423732 -2.69176877 -2.59720016 -2.64816323 -2.57859434
 -2.72405068 -2.54628464 -2.70420214 -2.69512981] [-2.649874   -2.71144665 -2.50350564 -2.63653292 -2.69972781 -2.62187785
 -2.61659659 -2.71313989 -2.66140395 -2.60745609]


In [7]:
A_x,B_y = np.ones(X),np.ones(Y)
maxIter = 1000
for i in range(maxIter):
    halfKB_x = K_x_y @ B_y / 2
    A_x = np.sqrt(halfKB_x*halfKB_x + n_x) - halfKB_x
    halfAK_y = A_x @ K_x_y / 2
    B_y = np.sqrt(halfAK_y*halfAK_y + m_y) - halfAK_y
    
a_x,b_y = 2*np.log(A_x), 2*np.log(B_y)
print(a_x,b_y)

[-2.62076699 -2.61421618 -2.69173806 -2.59717903 -2.6481351  -2.57857609
 -2.72401357 -2.54627086 -2.70416671 -2.69509666] [-2.64989955 -2.71146417 -2.50354489 -2.63656053 -2.69974544 -2.62190492
 -2.61662929 -2.7131581  -2.66142846 -2.60748781]


In [8]:
!pip install gurobipy --upgrade

In [9]:
import gurobipy as grb

In [10]:
grb.gurobi.version()

(11, 0, 2)

# Comparing the logit model in closed-form and simulated

In [11]:
X=Y=3
np.random.seed(7)
phi_x_y = np.random.uniform(size = (X,Y))
n_x,m_y = np.ones(X),np.ones(Y)
a_x,b_y = np.zeros(X),np.zeros(Y)
K_x_y = np.exp(phi_x_y / 2)
maxIter = 10000
eps = 0.2
tol = 1e-5
for i in range(maxIter):
    mu_x_y = K_x_y * np.exp( (a_x[:,None]+b_y[None,:])/2 )
    mu_x0 = np.exp(a_x)
    mu_0y = np.exp(b_y)
    der_x = mu_x0 + mu_x_y.sum( axis = 1) - n_x 
    der_y = mu_0y + mu_x_y.sum( axis = 0) - m_y
    if max(np.max(np.abs(der_x / n_x)),np.max(np.abs(der_y / m_y)))<tol:
        break
    a_x -= eps * der_x
    b_y -= eps * der_y

print(i)
print(a_x,b_y)
u_x = np.log(n_x) - a_x
v_y = np.log(m_y) - b_y
print(u_x,v_y)

105
[-1.54590499 -1.75196975 -1.44910952] [-1.54253516 -1.66437991 -1.52266072]
[1.54590499 1.75196975 1.44910952] [1.54253516 1.66437991 1.52266072]


In [12]:
import math as m
EulerMascheroniApp = round( (1.-m.gamma(1+1.e-8))*1.e14 )*1.e-6
print(EulerMascheroniApp)

0.577216


In [13]:
B=10000
epsilon_b_x_y = - np.log ( (- np.log (np.random.uniform(size= (B,X,Y))))) - EulerMascheroniApp
epsilon_b_x_0 = - np.log ( (- np.log (np.random.uniform(size= (B,X)) ))) - EulerMascheroniApp
eta_x_b_y =  - np.log ( (- np.log ( np.random.uniform(size= (X,B,Y)) ))) - EulerMascheroniApp
eta_0_b_y =  - np.log ( (- np.log ( np.random.uniform(size= (B,Y))))) - EulerMascheroniApp

m = grb.Model()
m.Params.outputFlag = 0
u_b_x = m.addMVar((B,X) , lb = -grb.GRB.INFINITY)
v_b_y = m.addMVar((B,Y), lb = -grb.GRB.INFINITY)
U_x_y = m.addMVar((X,Y), lb = -grb.GRB.INFINITY)
V_x_y = m.addMVar((X,Y), lb = -grb.GRB.INFINITY)
m.setObjective(u_b_x.sum() + v_b_y.sum(), sense = grb.GRB.MINIMIZE)
mu_x_y = m.addConstr(U_x_y + V_x_y == phi_x_y )
mu_b_x_y = m.addConstr(u_b_x[:,:,None] >= U_x_y[None,:,:] + epsilon_b_x_y )
mu_b_x_0 = m.addConstr(u_b_x >= epsilon_b_x_0 )
mu_x_b_y = m.addConstr(v_b_y[None,:,:] >= V_x_y[:,None,:] + eta_x_b_y )
mu_0_b_y = m.addConstr(v_b_y >= eta_0_b_y)
m.optimize()

u_x = u_b_x.X.sum(axis = 0) / B
v_y = v_b_y.X.sum(axis = 0) / B
print(u_x,v_y)              

[1.53954646 1.76952361 1.45232908] [1.54533807 1.65003448 1.50784163]


In [34]:
X,Y=3,4
n_x,m_y = np.ones(X),np.ones(Y)
np.random.seed(7)
phi_x_y = np.random.uniform(size = (X,Y))
maxIter = 10000
eps = 0.1
tol = 1e-8
U_x_y = np.zeros((X,Y))
for i in range(maxIter):
    muX_x_y =  n_x[:,None] * np.exp(U_x_y) / (1+ np.exp(U_x_y).sum(axis = 1))[:,None]
    V_x_y = phi_x_y - U_x_y
    muY_x_y = m_y[None,:] * np.exp(V_x_y) / (1+ np.exp(V_x_y).sum(axis = 0))[None,:]
    grad_x_y = muX_x_y - muY_x_y 
    if np.max(np.abs(grad_x_y))<tol:
        break
    U_x_y -= eps * grad_x_y

print(i)
u_x = np.log (1+ np.exp(U_x_y).sum(axis = 1))
v_y = np.log (1+ np.exp(V_x_y).sum(axis = 0))
print(u_x,v_y)

1714
[2.35256996 2.37753728 2.39518984] [1.10527755 1.17339119 1.1370997  1.14609608]


In [30]:
X,Y=3,4
n_x,m_y = np.ones(X),np.ones(Y)
np.random.seed(7)
phi_x_y = np.random.uniform(size = (X,Y))
n_x,m_y = np.ones(X),np.ones(Y)
a_x,b_y = np.zeros(X),np.zeros(Y)
K_x_y = np.exp(phi_x_y / 2)
maxIter = 10000
eps = 0.2
tol = 1e-8
for i in range(maxIter):
    mu_x_y = K_x_y * np.exp( (a_x[:,None]+b_y[None,:])/2 )
    mu_x0 = np.exp(a_x)
    mu_0y = np.exp(b_y)
    der_x = mu_x0 + mu_x_y.sum( axis = 1) - n_x 
    der_y = mu_0y + mu_x_y.sum( axis = 0) - m_y
    if max(np.max(np.abs(der_x / n_x)),np.max(np.abs(der_y / m_y)))<tol:
        break
    a_x -= eps * der_x
    b_y -= eps * der_y

print(i)
#print(a_x,b_y)
u_x = np.log(n_x) - a_x
v_y = np.log(m_y) - b_y
print(u_x,v_y)

385
[2.35257    2.37753733 2.39518989] [1.10527752 1.17339116 1.13709967 1.14609605]


#  Parametric estimation of the Choo-Siow model

In [53]:
import scipy.sparse as sp
from sklearn import linear_model
clf = linear_model.PoissonRegressor( fit_intercept=False ,alpha = 0)

X,Y,K=30,40,9
np.random.seed(7)
phi_x_y_k = np.random.uniform(size = (X,Y,K))
muhat_x_y = np.random.uniform(size = (X,Y))
muhat_x0 = np.random.uniform(size = (X))
muhat_0y = np.random.uniform(size = (Y))


A = X*Y+X+Y
w_a = np.array( X*Y*[2]+(X+Y)*[1] )
R11 = phi_x_y_k.reshape((-1,K))/ 2
R12 = sp.kron(sp.eye(X),np.ones((Y,1))) / 2
R13 = sp.kron(np.ones((X,1)),sp.eye(Y) ) / 2
R_a_r = sp.bmat( [[R11,  R12,   R13],
             [None , sp.eye(X),None],
             [None, None , sp.eye(Y)]])
muhat_a = np.concatenate( [muhat_x_y.flatten(),muhat_x0,muhat_0y] )


clf.fit(R_a_r, muhat_a, sample_weight=w_a )

lambda_k,a_x,b_y = clf.coef_[:K],clf.coef_[K:(K+X)],clf.coef_[(K+X):]
mupred_x_y = np.exp( (  (phi_x_y_k.reshape( (-1,K)) @ lambda_k).reshape((X,Y))  + a_x[:,None] + b_y[None,:] )/2 )

In [55]:
print(mupred_x_y.sum(axis = 1 )[:5])
print(muhat_x_y.sum(axis = 1 )[:5])

[18.81736987 17.93447067 18.77736696 17.33913638 20.948189  ]
[18.49852159 17.81547259 18.84292644 17.11113707 20.53590121]


In [56]:
print(mupred_x_y.sum(axis = 0 )[:5])
print(muhat_x_y.sum(axis = 0 )[:5])

[13.97073989 15.10818863 14.9122898  13.40713563 14.82945182]
[13.93637342 15.00026524 14.88717982 13.73307095 14.31040246]


In [57]:
(mupred_x_y[:,:,None] * phi_x_y_k).sum(axis = (0,1))

array([289.9141168 , 294.84973467, 285.93049814, 294.0073479 ,
       296.90170703, 300.40568371, 296.44361374, 296.32348269,
       302.81625256])

In [58]:
(muhat_x_y[:,:,None] * phi_x_y_k).sum(axis = (0,1))

array([289.91883686, 294.75035579, 285.83552398, 293.91134241,
       296.84858084, 300.29740981, 296.33699132, 296.24313155,
       302.75104074])

In [59]:
Nhouseholds = muhat_x_y.sum()+muhat_x0.sum()+muhat_0y.sum()
Nhouseholds

628.8258137198626

In [60]:
(muhat_x_y / Nhouseholds )[:5]

array([[8.29803378e-04, 3.81558040e-04, 3.83812371e-04, 9.53457059e-04,
        1.35200991e-03, 1.28982431e-04, 4.99159913e-04, 4.98420110e-04,
        3.89227894e-04, 1.09584367e-03, 3.01854503e-04, 9.84989324e-04,
        9.53616152e-04, 3.04626024e-04, 4.17604745e-04, 1.01242020e-03,
        9.14058767e-04, 2.03052558e-04, 4.57108954e-04, 5.10909229e-04,
        9.22766152e-04, 2.29840633e-04, 8.34647929e-04, 7.33490694e-04,
        1.27817974e-03, 1.45149722e-03, 9.90105946e-04, 1.04557983e-03,
        1.45915773e-04, 5.61975503e-04, 2.92056954e-04, 3.89054218e-05,
        2.90721710e-04, 1.00441092e-03, 1.50501779e-03, 1.47078392e-03,
        7.19845566e-04, 1.03322573e-03, 1.07701909e-03, 1.21905903e-03],
       [4.28164116e-04, 1.58104407e-04, 8.30711027e-04, 1.36667581e-04,
        1.46910849e-03, 2.59260468e-04, 7.42445829e-04, 1.53367198e-03,
        7.48589731e-04, 3.10133145e-05, 1.07128996e-03, 1.18275134e-03,
        2.29845332e-04, 1.17081327e-03, 1.25500394e-03, 2.52950